In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("visual-question-answering", model="Salesforce/blip-vqa-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.54G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

The model 'BlipForQuestionAnswering' is not supported for visual-question-answering. Supported models are ['Blip2ForConditionalGeneration', 'ViltForQuestionAnswering'].


In [ ]:
import requests
from PIL import Image
from transformers import BlipProcessor, BlipForQuestionAnswering

processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")
model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base").to("cuda")

img_url = 'https://gampa.in/assets/images/banner/Gampa-2.jpg'
raw_image = Image.open(requests.get(img_url, stream=True).raw).convert('RGB')

question = "how many dogs are in image"
inputs = processor(raw_image, question, return_tensors="pt").to("cuda")

out = model.generate(**inputs)
print(processor.decode(out[0], skip_special_tokens=True))


0


In [ ]:

question = "what the dress code he wear"
inputs = processor(raw_image, question, return_tensors="pt").to("cuda")

out = model.generate(**inputs)
print(processor.decode(out[0], skip_special_tokens=True))


business


In [ ]:
# First, install the necessary libraries
!pip install gradio transformers torch pillow

# Import necessary libraries
import gradio as gr
from PIL import Image
from transformers import BlipProcessor, BlipForQuestionAnswering
import torch

# Load the processor and model from Hugging Face Hub
processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")
model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base")

# Define a function that uses the model to answer questions about the image
def answer_question(image, question):
    # Convert the image to RGB
    image = image.convert('RGB')

    # Prepare inputs
    inputs = processor(image, question, return_tensors="pt")

    # Send model inputs to CUDA if available
    if torch.cuda.is_available():
        inputs = inputs.to("cuda")
        model.to("cuda")

    # Generate an answer
    output = model.generate(**inputs)

    # Decode the generated tokens to text
    answer = processor.decode(output[0], skip_special_tokens=True)
    return answer

# Create a Gradio interface
interface = gr.Interface(
    fn=answer_question,
    inputs=[
        gr.components.Image(type="pil"),
        gr.components.Textbox(label="Question about the image")
    ],
    outputs=gr.components.Textbox(),
    title="Visual Question Answering",
    description="Upload an image and ask a question about it."
)

# Launch the Gradio app
interface.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://84e59c8a717a838fa6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
